# KERAS-TF2.0-FaceAttribute

## 1. Dataset

In [22]:
import tensorflow as tf
import numpy as np
import random
import os

### Load labels

In [57]:
def load_labels(label_file):
    f = open(label_file)
    # line 1: number of images
    num_imgs = int(f.readline())
    # line 2: attribute names, 40 in total
    attr_names = f.readline().split()
    # line 3 to end: 00xx.jpg -1 1 -1 1 ...
    labels = []
    for i in range(num_imgs):
        labels.append(list(map(np.float32, f.readline().split()[1:])))
    labels = np.array(labels)
    labels[labels<0] = 0
    return labels

In [58]:
labels = load_labels('../dataset/list_attr_celeba.txt')

### Load images

In [62]:
image_paths = os.listdir('../dataset/img_align_celeba')
image_paths.sort()
image_count = len(image_paths)

### Build dataset

In [52]:
def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    # need to resize?
    img = tf.image.resize(img, [218, 178])
    # img = img/128.0
    return img, label

In [61]:
celeba_ds = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(load_and_preprocess)

In [65]:
BATCH_SIZE = 32
ds = celeba_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## 2. Model